In [0]:
# please save me I want to have some sleep
import pandas as pd
import numpy as np
import tensorflow as tf
import mlflow
import mlflow.tensorflow
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import Model
import os
import tempfile

In [0]:
# 2. authenticate and experiment

# Authenticate with Azure Managed Identity automatically
credential = DefaultAzureCredential()

# Connect to Azure ML
ml_client = MLClient(
    credential=credential,
    subscription_id="0a601663-6ddf-4fa7-9c07-cf51438535af",
    resource_group_name="diplomka",
    workspace_name="AML-diplomka"
)

print("Connected to Azure ML workspace")

# initiate ml flow experiment for saving training data
mlflow.set_experiment("/Users/bulanec123@gmail.com/irradiance_retrain_experiment")

model_name = "irradiance"

In [0]:
# 3. functions (all logic to use)

def load_latest_model(model_name, ml_client):
    models = list(ml_client.models.list(name=model_name))

    # get latest model was not able to figure it out from documentation done by chat gpt
    latest_model = sorted(models, key=lambda m: m.creation_context.created_at, reverse=True)[0]

    print(f"Latest model: {latest_model.name}, version: {latest_model.version}")

    # get the latest model to dfbs so we can access it for training we will delete if afterwards
    download_path = "./local_model"
    ml_client.models.download(
        name=latest_model.name,
        version=latest_model.version,
        download_path=download_path
    )
    model_folder = os.path.join(download_path, latest_model.name)
    keras_model_path = None

    # was not able to find it with path sometimes so I go through each one
    for root, dirs, files in os.walk(model_folder):
        for file in files:
            if file.endswith(".keras"):
                keras_model_path = os.path.join(root, file)
                break


    model = tf.keras.models.load_model(keras_model_path)
    print(f"Model loaded")
    return model, latest_model

def load_all_gold_tables():
    tables = spark.catalog.listTables("model_workspace.gold")
    table_names = [t.name for t in tables if t.tableType == "MANAGED"]
    table_names.sort()
    print(f"najdene: {table_names}")

    train_dfs, val_dfs = [], []

    for idx, table_name in enumerate(table_names):
        df = spark.read.table(f"model_workspace.gold.{table_name}").toPandas()
        
        # if 'UnixTime' in df.columns:
        #     df = df.drop(columns=['UnixTime'])

        # Currently not checking for new data but importing all available data in gold tables and retraining the model
        # This was done in this way because we do not have more available data for training and this solution is proof of concept 
        # not a production environment, this was done for testing as well

        if idx < 3:
            train_dfs.append(df)
        else:
            val_dfs.append(df)

    train_df = pd.concat(train_dfs, ignore_index=True)
    val_df = pd.concat(val_dfs, ignore_index=True)
    print(f"train : {len(train_df)}   val : {len(val_df)}")
    return train_df, val_df

def preprocess_data(df):
    
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(df)
    
    df_scaled = pd.DataFrame(scaled_data, columns=df.columns)
    return df_scaled, scaler

def create_sequences(df_scaled, target_column_name="Irradiance", seq_length=48, use_multivariate=True):
    data = df_scaled.values
    target_column = df_scaled.columns.get_loc(target_column_name)
    X, y = [], []
    for i in range(len(data) - seq_length):
        if not use_multivariate:
            X.append(data[i:i + seq_length, target_column].reshape(-1, 1))
        else:
            X.append(data[i:i + seq_length, :])
        y.append(data[i + seq_length, target_column])
    return np.array(X), np.array(y)

def fine_tune_model(model, X_train, y_train, X_val, y_val, learning_rate=0.00000001, batch_size=32, epochs=5):
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train,epochs=epochs,batch_size=batch_size,validation_data=(X_val, y_val),callbacks=[early_stopping],verbose=1)
    val_loss = history.history['val_loss'][-1]
    return model, val_loss

def compare_and_register(model_name, ml_client, model, val_loss, old_model, X_val, y_val, scaler, df_scaled_val):
    y_pred_scaled = model.predict(X_val)
    target_index = df_scaled_val.columns.get_loc("Irradiance")

    # need to inverse to get real data for valdating it
    y_val_actual = inverse_transform_target(y_val, scaler, df_scaled_val.shape[1], target_index)
    y_pred_actual = inverse_transform_target(y_pred_scaled, scaler, df_scaled_val.shape[1], target_index)

   
    fine_tuned_r2 = r2_score(y_val_actual, y_pred_actual)
    print(f"fine-tuned   R2: {fine_tuned_r2:.4f}")

    # get old metrics from tags that we inseted in the model in Azure ML we can see it in picture in thesis as well
    old_val_loss = old_model.tags.get("val_loss")
    old_r2 = old_model.tags.get("r2_score")
    old_val_loss = float(old_val_loss)
    old_r2 = float(old_r2)
        

    is_better = False
    if (old_val_loss is None) or (val_loss < old_val_loss):
        is_better = True
    elif (old_r2 is None) or (fine_tuned_r2 > old_r2):
        is_better = True
   

    # register model if its prediction was better then old one

    if is_better:
        with tempfile.TemporaryDirectory() as tmpdir:
            model_save_path = os.path.join(tmpdir, "model.keras")
            model.save(model_save_path)

            registered_model = Model(
                path=tmpdir,
                name=model_name,
                description="Fine-tuned model with improved validation loss or R2",
                tags={
                    "val_loss": str(val_loss),
                    "r2_score": str(fine_tuned_r2)
                }
            )
            registered_model = ml_client.models.create_or_update(registered_model)
            print(f"registered: {registered_model.name}, version: {registered_model.version}")
    else:
        print("did not improve")

def inverse_transform_target(scaled_target, scaler, n_features, target_index):
    zero_filled = np.zeros((len(scaled_target), n_features))
    zero_filled[:, target_index] = scaled_target.flatten()
    return scaler.inverse_transform(zero_filled)[:, target_index]

def plot_and_log_predictions(X_val, y_val, model, scaler, df_scaled_val):
    y_pred_scaled = model.predict(X_val)
    target_index = df_scaled_val.columns.get_loc("Irradiance")

    y_val_actual = inverse_transform_target(y_val, scaler, df_scaled_val.shape[1], target_index)
    y_pred_actual = inverse_transform_target(y_pred_scaled, scaler, df_scaled_val.shape[1], target_index)

    predictions_df = pd.DataFrame({
        "Actual Irradiance": y_val_actual,
        "Predicted Irradiance": y_pred_actual
    })


    fine_tuned_r2 = r2_score(y_val_actual, y_pred_actual)
    fine_tuned_mae = mean_absolute_error(y_val_actual, y_pred_actual)
    fine_tuned_rmse = np.sqrt(mean_squared_error(y_val_actual, y_pred_actual))

    # create a temp folder
    with tempfile.TemporaryDirectory() as tmpdir:
        # --- Save plot ---
        plots_dir = os.path.join(tmpdir, "plots")
        os.makedirs(plots_dir, exist_ok=True)
        plot_path = os.path.join(plots_dir, "actual_vs_predicted.png")

        plt.figure(figsize=(14, 6))
        plt.plot(predictions_df["Actual Irradiance"], label="Actual", alpha=0.7)
        plt.plot(predictions_df["Predicted Irradiance"], label="Predicted", alpha=0.7)
        plt.title("Validation Set: Actual vs Predicted Irradiance")
        plt.xlabel("Time Step")
        plt.ylabel("Irradiance")
        plt.legend()
        plt.grid(True)
        plt.savefig(plot_path)
        plt.close()

        mlflow.log_artifact(plot_path, artifact_path="plots")
        print(f"plot saved")

        # save metrics to experiment
        metrics_dir = os.path.join(tmpdir, "metrics")
        os.makedirs(metrics_dir, exist_ok=True)
        metrics_path = os.path.join(metrics_dir, "metrics.json")

        metrics = {
            "r2_score": fine_tuned_r2,
            "mae": fine_tuned_mae,
            "rmse": fine_tuned_rmse
        }
        with open(metrics_path, "w") as f:
            json.dump(metrics, f, indent=4)

        mlflow.log_artifact(metrics_path, artifact_path="metrics")
        print(f"metrics file saved")

    mlflow.log_metric("r2_score", fine_tuned_r2)
    mlflow.log_metric("mae", fine_tuned_mae)
    mlflow.log_metric("rmse", fine_tuned_rmse)

    print(f"R2: {fine_tuned_r2:.4f}")
    print(f"MAE: {fine_tuned_mae:.4f}")
    print(f"RMSE: {fine_tuned_rmse:.4f}")

In [0]:
# 4. main code of this task

with mlflow.start_run(run_name="Retrain-FineTune-Register"):

    mlflow.log_param("model_name", model_name)
    mlflow.set_tag("pipeline", "automated_retrain")

    model, old_model = load_latest_model(model_name, ml_client)
    train_df, val_df = load_all_gold_tables()

    train_scaled, scaler = preprocess_data(train_df)
    val_scaled, _ = preprocess_data(val_df)

    X_train, y_train = create_sequences(train_scaled)
    X_val, y_val = create_sequences(val_scaled)

    fine_tuned_model, fine_tuned_val_loss = fine_tune_model(model, X_train, y_train, X_val, y_val)

    mlflow.log_metric("fine_tuned_val_loss", fine_tuned_val_loss)

    compare_and_register(model_name, ml_client, fine_tuned_model, fine_tuned_val_loss, old_model, X_val, y_val, scaler, val_scaled)
    plot_and_log_predictions(X_val, y_val, fine_tuned_model, scaler, val_scaled)

print("workflow is finished")